# Data loaders

In PyTorch, the data loader plays an indispensable role in managing this vast amount of data. For natural language processing (NLP) tasks like yours, data often comes in variable lengths due to differing sentence structures and lengths across languages. The data loader efficiently batches these variable-length sequences, ensuring that your models are trained on diverse examples in every iteration. This batching is crucial for harnessing the power of parallel computation on GPUs, thus expediting the training process.

Furthermore, the data loader aids in shuffling the data set, which is vital for preventing models from memorizing the sequence of training data and promoting better generalization. Especially for NLP tasks, where data might be ordered or clustered by topics, shuffling ensures that the model remains robust and doesn't develop biases based on the order of input.

Lastly, in the world of NLP, preprocessing steps such as tokenization, padding, and numericalization are paramount. The data loader in PyTorch provides hooks that allow us to seamlessly integrate these preprocessing steps, ensuring that the raw textual data is transformed into a format that's amenable for deep learning models. 

### parameters

Data loaders have several key parameters, including the data set to load from, batch size (determining how many samples per batch), shuffle (whether to shuffle the data for each epoch), and more. Data loaders also provide an iterator interface, making it easy to iterate over batches of data during training.

Iterators are commonly used to traverse large data sets without loading all elements into memory simultaneously, making the process more memory-efficient. In PyTorch, not all data sets are iterators, but all data loaders are.

In PyTorch, the data loader processes data in batches, loading and processing one batch at a time into memory efficiently. The batch size, which you specify when creating the data loader, determines how many samples are processed together in each batch. **The data loader's purpose is to convert input data and labels into batches of tensors with the same shape for deep learning models to interpret.**



Finally, a data loader can be used for tasks such as tokenizing, sequencing, converting your samples to the same size, and transforming your data into tensors that your model can understand.

## **Custom data set and data loader in PyTorch**
In this code snippet, you will see how to create a custom data set and use the DataLoader class in PyTorch. The data set consists of a list of random sentences, and the objective is to create batches of sentences for further processing, such as training a neural network model.

You will begin by defining a custom data set called CustomDataset. This data set inherits from the `torch.utils.data.Dataset` class and is initialized with a list of sentences. The data set comprises two essential methods:

- __init__(self, sentences): Initializes the data set with a list of sentences.
- __getitem__(self, idx): Retrieves an item (in this case, a sentence) at a specific index, idx.

Next, you create an instance of your custom data set (custom_dataset) by passing in the list of sentences. Additionally, you can specify a batch size (batch_size), which determines how many sentences will be grouped together in each batch during data loading.

You will then create a DataLoader (dataloader) by providing your custom data set and batch size to the torch.utils.data.DataLoader class. Furthermore, you set shuffle=True, indicating that the sentences will be randomly shuffled before being divided into batches. This shuffling is particularly useful for training deep learning models, as it prevents the model from learning patterns based on the order of the data.

Finally, you iterate through the DataLoader to demonstrate how data is loaded in batches. In this code, you will see that batch size is set to 2, meaning that each batch will contain two sentences. The DataLoader efficiently manages the loading of data in batches, making it suitable for training deep learning models.

During iteration, the sentences in each batch are printed to illustrate how the DataLoader groups and presents the data. This code snippet provides a fundamental example of how to set up a custom data set and data loader in PyTorch, which is a common practice in deep learning workflows.


In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torchtext

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.datapipes.iter import IterableWrapper
import numpy as np
import random

/opt/homebrew/Caskroom/mambaforge/base/envs/spacy-fr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

# Create an instance of your custom dataset
custom_dataset = CustomDataset(sentences)

# Define batch size
batch_size = 2

# Create a DataLoader
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterate through the DataLoader
for batch in dataloader:
    print(batch)

["If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.", 'Soon we must all face the choice between what is right and what is easy.']
['Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.', 'You are awesome!']
['It is our choices, Harry, that show what we truly are, far more than our abilities.', "Fame's a fickle friend, Harry."]



As shown above, the data is organized into batches of 2 sentences each. It's important to note that deep learning models can only comprehend numerical data, and words are meaningless to them. Therefore, your next step is to convert these sentences into tensors. Let's see how to do this.

### Creating tensors for custom data set

In this code example, you will see the creation of a custom data set for natural language processing (NLP) tasks using PyTorch. The data set consists of a list of sentences, and your goal is to preprocess these sentences, tokenize them, and convert them into tensors of token indices for use in NLP models. Let's break down the code step by step.

The sentences and the CustomDataset class are used in the same way as in the previous code snippet. The changes made to the CustomDataset class are as follows:

- __init__: The constructor takes a list of sentences, a tokenizer function, and a vocabulary (vocab) as input.
- __len__: This method returns the total number of samples in the data set.
- __getitem__: This method is responsible for processing a single sample. It tokenizes the sentence using the provided tokenizer and then converts the tokens into tensor indices using the vocabulary.

You can define a tokenizer using the `get_tokenizer` function with the `basic_english` option. Tokenization is the process of splitting a text into individual tokens or words. Next, you build a vocabulary from the sentences. You use the `build_vocab_from_iterator` function to construct the vocabulary from the tokenized sentences.

You can create an instance of your custom data set, passing in the sentences, tokenizer, and vocabulary. Finally, you print the length of the custom data set and sample items from the data set for illustration.


In [3]:
sentences = [
    "If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
    "Fame's a fickle friend, Harry.",
    "It is our choices, Harry, that show what we truly are, far more than our abilities.",
    "Soon we must all face the choice between what is right and what is easy.",
    "Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.",
    "You are awesome!"
]

# Define a custom data set
class CustomDataset(Dataset):
    def __init__(self, sentences, tokenizer, vocab):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.vocab = vocab

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.sentences[idx])
        # Convert tokens to tensor indices using vocab
        tensor_indices = [self.vocab[token] for token in tokens]
        return torch.tensor(tensor_indices)

# Tokenizer
tokenizer = get_tokenizer("basic_english")

# Build vocabulary
vocab = build_vocab_from_iterator(map(tokenizer, sentences))

# Create an instance of your custom data set
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

print("Custom Dataset Length:", len(custom_dataset))
print("Sample Items:")
for i in range(6):
    sample_item = custom_dataset[i]
    print(f"Item {i + 1}: {sample_item}")

Custom Dataset Length: 6
Sample Items:
Item 1: tensor([11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
        43, 61,  9, 44,  0, 14,  9, 33,  1])
Item 2: tensor([35,  6, 16,  3, 38, 40,  0,  8,  1])
Item 3: tensor([12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
        21,  1])
Item 4: tensor([54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1])
Item 5: tensor([66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
         2, 12, 64, 17, 26, 65,  1])
Item 6: tensor([19,  4, 25, 20])


In [4]:
# Create an instance of your custom data set
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

# Define batch size
batch_size = 2

# Create a data loader
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterate through the data loader
for batch in dataloader:
    print(batch)

RuntimeError: stack expects each tensor to be equal size, but got [16] at entry 0 and [4] at entry 1

You will encounter an error when attempting to create batches for the tensors. This error arises because the tensor batches have unequal lengths. The data loader is using the default `collate_function`, which requires tensors to have equal lengths. You can define your own `collate_function` and pass the data into it to establish your own rules. Typically, to address the issue of unequal tensor lengths, you employ data padding. This will be demonstrated in the following section.


### Custom collate function

A collate function is employed in the context of data loading and batching in machine learning, particularly when dealing with variable-length data, such as sequences (e.g., text, time series, sequences of events). Its primary purpose is to prepare and format individual data samples (examples) into batches that can be efficiently processed by machine learning models.

You will begin by defining a custom collate function named `collate_fn`. This function plays a crucial role when handling sequences of varying lengths, such as sentences in NLP. Its purpose is to pad sequences within a batch to have equal lengths, which is a common preprocessing step in NLP tasks.

`pad_sequence`: This function is a part of PyTorch and is utilized to pad sequences in a batch, ensuring uniform length. It takes a batch of sequences as input and pads them to match the length of the longest sequence. The `padding_value=0` argument specifies the value to use for padding.


In [ ]:
# Create a custom collate function
def collate_fn(batch):
    # Pad sequences within the batch to have equal lengths
    padded_batch = pad_sequence(batch, batch_first=True, padding_value=0)
    return padded_batch

In the above cell, when padding the sequences, you set `batch_first=True`. When `batch_first=True`, output will be in [batch_size x seq_len] shape, otherwise it will be in [seq_len x batch_size] shape. Some models accept input with [batch_size x seq_len] shape while some other models need the input to be of [seq_len x batch_size] shape. Keep in mind that this parameter takes care of putting the input in the desired shape. 

Let's see how it actually affects the shape of curated batches. First, you create a data loader from a collate function with `batch_first=True`:


In [ ]:
# Create a data loader with the custom collate function with batch_first=True,
dataloader = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Iterate through the data loader
for batch in dataloader: 
    for row in batch:
        for idx in row:
            words = [vocab.get_itos()[idx] for idx in row]
        print(words)
       

['if', 'you', 'want', 'to', 'know', 'what', 'a', 'man', "'", 's', 'like', ',', 'take', 'a', 'good', 'look', 'at', 'how', 'he', 'treats', 'his', 'inferiors', ',', 'not', 'his', 'equals', '.']
['fame', "'", 's', 'a', 'fickle', 'friend', ',', 'harry', '.', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',']
['it', 'is', 'our', 'choices', ',', 'harry', ',', 'that', 'show', 'what', 'we', 'truly', 'are', ',', 'far', 'more', 'than', 'our', 'abilities', '.']
['soon', 'we', 'must', 'all', 'face', 'the', 'choice', 'between', 'what', 'is', 'right', 'and', 'what', 'is', 'easy', '.', ',', ',', ',', ',']
['youth', 'can', 'not', 'know', 'how', 'age', 'thinks', 'and', 'feels', '.', 'but', 'old', 'men', 'are', 'guilty', 'if', 'they', 'forget', 'what', 'it', 'was', 'to', 'be', 'young', '.']
['you', 'are', 'awesome', '!', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',']


Looking into the result, you can see that the first dimension is the batch. For example, first batch is the first sentence: "['if', 'you', 'want', 'to', 'know', 'what', 'a', 'man', "'", 's', 'like', ',', 'take', 'a', 'good', 'look', 'at', 'how', 'he', 'treats', 'his', 'inferiors', ',', 'not', 'his', 'equals', '.']". 

Now, you can try `batch_first=False` which is the DEFAULT value:


In [ ]:
# Create a custom collate function
def collate_fn_bfFALSE(batch):
    # Pad sequences within the batch to have equal lengths
    padded_batch = pad_sequence(batch, padding_value=0)
    return padded_batch

In [ ]:
custom_dataset

In [ ]:


# Create a data loader with the custom collate function with batch_first=True,
dataloader_bfFALSE = DataLoader(custom_dataset, batch_size=batch_size, collate_fn=collate_fn_bfFALSE)

# Iterate through the data loader
for seq in dataloader_bfFALSE:
    for row in seq:
        #print(row)
        words = [vocab.get_itos()[idx] for idx in row]
        print(words)

['if', 'fame']
['you', "'"]
['want', 's']
['to', 'a']
['know', 'fickle']
['what', 'friend']
['a', ',']
['man', 'harry']
["'", '.']
['s', ',']
['like', ',']
[',', ',']
['take', ',']
['a', ',']
['good', ',']
['look', ',']
['at', ',']
['how', ',']
['he', ',']
['treats', ',']
['his', ',']
['inferiors', ',']
[',', ',']
['not', ',']
['his', ',']
['equals', ',']
['.', ',']
['it', 'soon']
['is', 'we']
['our', 'must']
['choices', 'all']
[',', 'face']
['harry', 'the']
[',', 'choice']
['that', 'between']
['show', 'what']
['what', 'is']
['we', 'right']
['truly', 'and']
['are', 'what']
[',', 'is']
['far', 'easy']
['more', '.']
['than', ',']
['our', ',']
['abilities', ',']
['.', ',']
['youth', 'you']
['can', 'are']
['not', 'awesome']
['know', '!']
['how', ',']
['age', ',']
['thinks', ',']
['and', ',']
['feels', ',']
['.', ',']
['but', ',']
['old', ',']
['men', ',']
['are', ',']
['guilty', ',']
['if', ',']
['they', ',']
['forget', ',']
['what', ',']
['it', ',']
['was', ',']
['to', ',']
['be', ',']
['

It can be seen that the first dimension is now the sequence instead of batch, which means sentences will break so that each row includes a token from each sequence. For example the first row, (['if', 'fame']), includes the first tokens of all the sequences in that batch. You need to be aware of this standard to avoid any confusion when working with recurrent neural networks (RNNs) and transformers.


In [ ]:
# Iterate through the data loader with batch_first = TRUE
for batch in dataloader:    
    print(batch)
    print("Length of sequences in the batch:",batch.shape[1])

tensor([[11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
         43, 61,  9, 44,  0, 14,  9, 33,  1],
        [35,  6, 16,  3, 38, 40,  0,  8,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0]])
Length of sequences in the batch: 27
tensor([[12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
         21,  1],
        [54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1,  0,  0,
          0,  0]])
Length of sequences in the batch: 20
tensor([[66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
          2, 12, 64, 17, 26, 65,  1],
        [19,  4, 25, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0]])
Length of sequences in the batch: 25


You will see that each batch has a fixed size for all the sequences within the batch.

You also have the option to utilize the collate function for tasks such as tokenization, converting tokenized indices, and transforming the result into a tensor. It's important to note that the original data set remains untouched by these transformations.


In [ ]:
# Define a custom data set
class CustomDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]

In [ ]:
sentences

["If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals.",
 "Fame's a fickle friend, Harry.",
 'It is our choices, Harry, that show what we truly are, far more than our abilities.',
 'Soon we must all face the choice between what is right and what is easy.',
 'Youth can not know how age thinks and feels. But old men are guilty if they forget what it was to be young.',
 'You are awesome!']

In [ ]:
custom_dataset = CustomDataset(sentences)
custom_dataset[0]

"If you want to know what a man's like, take a good look at how he treats his inferiors, not his equals."

In [ ]:
def collate_fn(batch):
    # Tokenize each sample in the batch using the specified tokenizer
    tensor_batch = []
    for sample in batch:
        tokens = tokenizer(sample)
        # Convert tokens to vocabulary indices and create a tensor for each sample
        tensor_batch.append(torch.tensor([vocab[token] for token in tokens]))
    # Pad sequences within the batch to have equal lengths using pad_sequence
    # batch_first=True ensures that the tensors have shape (batch_size, max_sequence_length)
    padded_batch = pad_sequence(tensor_batch, batch_first=True)
    
    # Return the padded batch
    return padded_batch

In [ ]:
# Create a data loader for the custom dataset
dataloader = DataLoader(
    dataset=custom_dataset,   # Custom PyTorch Dataset containing your data
    batch_size=batch_size,     # Number of samples in each mini-batch
    shuffle=True,              # Shuffle the data at the beginning of each epoch
    collate_fn=collate_fn      # Custom collate function for processing batches
)

You will see that the result is a tensor of the same shape for each sample in the batch.


In [ ]:
for batch in dataloader:
    print(batch)
    print("shape of sample",len(batch))

tensor([[12,  5, 15, 31,  0,  8,  0, 57, 53,  2, 18, 62,  4,  0, 36, 49, 56, 15,
         21,  1,  0,  0,  0,  0,  0],
        [66, 29, 14, 13, 10, 22, 60,  7, 37,  1, 28, 51, 48,  4, 42, 11, 59, 39,
          2, 12, 64, 17, 26, 65,  1]])
shape of sample 2
tensor([[35,  6, 16,  3, 38, 40,  0,  8,  1,  0,  0,  0,  0,  0,  0,  0],
        [54, 18, 50, 23, 34, 58, 30, 27,  2,  5, 52,  7,  2,  5, 32,  1]])
shape of sample 2
tensor([[11, 19, 63, 17, 13,  2,  3, 47,  6, 16, 45,  0, 55,  3, 41, 46, 24, 10,
         43, 61,  9, 44,  0, 14,  9, 33,  1],
        [19,  4, 25, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0]])
shape of sample 2


As a result, batches of tensors with equal lengths have been successfully created.


### Exercise 

Create a data lodaer with a collate func that processes batches of french text. Sort the data set on sequences length. Then, tokenize, numericalize and pad the sequences. 

Sorting the sequences will minimize the number of PAD tokens added to the sequences, which enhances the model's performance.

Prepare the data in batches of size 4 and print them

In [ ]:
corpus = [
    "Ceci est une phrase.",
    "C'est un autre exemple de phrase.",
    "Voici une troisième phrase.",
    "Il fait beau aujourd'hui.",
    "J'aime beaucoup la cuisine française.",
    "Quel est ton plat préféré ?",
    "Je t'adore.",
    "Bon appétit !",
    "Je suis en train d'apprendre le français.",
    "Nous devons partir tôt demain matin.",
    "Je suis heureux.",
    "Le film était vraiment captivant !",
    "Je suis là.",
    "Je ne sais pas.",
    "Je suis fatigué après une longue journée de travail.",
    "Est-ce que tu as des projets pour le week-end ?",
    "Je vais chez le médecin cet après-midi.",
    "La musique adoucit les mœurs.",
    "Je dois acheter du pain et du lait.",
    "Il y a beaucoup de monde dans cette ville.",
    "Merci beaucoup !",
    "Au revoir !",
    "Je suis ravi de vous rencontrer enfin !",
    "Les vacances sont toujours trop courtes.",
    "Je suis en retard.",
    "Félicitations pour ton nouveau travail !",
    "Je suis désolé, je ne peux pas venir à la réunion.",
    "À quelle heure est le prochain train ?",
    "Bonjour !",
    "C'est génial !"
]



In [ ]:
!pip install "numpy<2"


In [ ]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 1.5 MB/s eta 0:00:0000:0100:03
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [ ]:
class frenchDataset(Dataset):
    def __init__(self, sentences, tokenizer, vocab):
        self.sentences = sentences
        self.tokenizer = tokenizer 
        self.vocab = vocab 
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        #return self.sentences[idx]
        tokens = self.tokenizer(self.sentences[idx])
        tensor_indexes = [self.vocab[token] for token in tokens]
        return torch.tensor(tensor_indexes)


tokenizer = get_tokenizer("spacy", language="fr_core_news_sm")

vocab = build_vocab_from_iterator(map(tokenizer, corpus))
vocab.set_default_index(vocab["<unk>"])
vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)

french_dataset = frenchDataset(corpus, tokenizer, vocab)

batch_size = 4

print("Custom Dataset Length:", len(french_dataset))
print("Sample Items:")
for i in range(6):
    sample_item = french_dataset[i]
    print(f"Item {i + 1}: {sample_item}")

Custom Dataset Length: 30
Sample Items:
Item 1: tensor([28,  4, 10,  9,  0])
Item 2: tensor([ 11,   4, 111,  50,  68,   5,   9,   0])
Item 3: tensor([ 38,  10, 107,   9,   0])
Item 4: tensor([12, 69, 51, 49,  0])
Item 5: tensor([31, 43,  8, 15, 57, 73,  0])
Item 6: tensor([37,  4, 19, 92, 95,  7])


In [ ]:
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: len(x), reverse=True)
    return pad_sequence(batch, batch_first=True, padding_value=0)



dataloader_fr = DataLoader(french_dataset, batch_size=4, collate_fn=collate_fn, shuffle=True)


for batch in dataloader_fr:
    print("shape of sample",len(batch))
    for row in batch:
        words = [vocab.get_itos()[idx] for idx in row]
        print(words)
    print("*"*20)

shape of sample 4
['Je', 'suis', 'en', 'train', "d'", 'apprendre', 'le', 'français', '.']
["C'", 'est', 'un', 'autre', 'exemple', 'de', 'phrase', '.', '.']
['Nous', 'devons', 'partir', 'tôt', 'demain', 'matin', '.', '.', '.']
['Au', 'revoir', '!', '.', '.', '.', '.', '.', '.']
********************
shape of sample 4
['Je', 'suis', 'désolé', ',', 'je', 'ne', 'peux', 'pas', 'venir', 'à', 'la', 'réunion', '.']
['À', 'quelle', 'heure', 'est', 'le', 'prochain', 'train', '?', '.', '.', '.', '.', '.']
['Ceci', 'est', 'une', 'phrase', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['Je', "t'", 'adore', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
********************
shape of sample 4
['Il', 'fait', 'beau', "aujourd'hui", '.']
["C'", 'est', 'génial', '!', '.']
['Je', 'suis', 'là', '.', '.']
['Bonjour', '!', '.', '.', '.']
********************
shape of sample 4
['Il', 'y', 'a', 'beaucoup', 'de', 'monde', 'dans', 'cette', 'ville', '.']
['La', 'musique', 'adoucit', 'les', 'mœurs', '.', '.', '.'